In [477]:
import pandas as pd
import numpy as np

In [478]:
amazon = pd.read_csv("amazon.csv")

In [479]:
amazon.dtypes

Name            object
Author          object
User Rating    float64
Reviews          int64
Price           object
Price_r          int64
Year             int64
Genre           object
dtype: object

In [480]:
amazon["Price_r"] = amazon.apply( lambda r: r["Price_r"] if r["Price_r"] >= 0 else r["Price_r"]*-1 , axis = 1 )

In [481]:
amazon

,Name,Author,User Rating,Reviews,Price,Price_r,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,$8.00,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,$22.00,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,$15.00,15,2018,Non Fiction
3,1984,George Orwell,4.7,70425,$7.48,8,2021,Fiction
4,1984 (Signet Classics),George Orwell,4.7,21424,$6.00,6,2017,Fiction
...,...,...,...,...,...,...,...,...
645,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,$8.00,8,2019,Fiction
646,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,$8.00,8,2016,Non Fiction
647,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,$8.00,8,2017,Non Fiction
648,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,$8.00,8,2018,Non Fiction


## Preguntas

Por año cuantos libros ha publicado un autor, recuerde que los años no trabajados deberia de aparecer en cero. considere como primer registro por autor el primer año en que publico y el ultimo el año maximo encontrado.

Ejemplo, si Uayeb public en 2009 dos libros, y en 2011 tres. y esas son las dos unicas entradas para este autor. la salida para el deberia de ser:

In [482]:
pd.DataFrame({ "Autor" : ["Uayeb","Uayeb","Uayeb"] , "Year" : [2009,2010,2011]  , "Conteo" : [2,0,3] })

,Autor,Year,Conteo
0,Uayeb,2009,2
1,Uayeb,2010,0
2,Uayeb,2011,3


In [483]:
Autor = amazon.loc[: , ["Author","Year","Name"]].sort_values(["Author"], ascending = True)

In [484]:
Autor = Autor.groupby(["Author","Year"], as_index=False).agg({"Name" : ["count"]})
Autor.columns = Autor.columns.droplevel(1)

In [485]:
YearTotal = amazon.loc[: , ["Year"]].drop_duplicates().sort_values(["Year"], ascending = True)


In [486]:
YearTotal['Indice'] = range(0,YearTotal.shape[0])


In [487]:
Autor = Autor.merge(YearTotal, on = ["Year"] , how = "inner")

In [488]:
Autor_range = Autor.groupby(["Author"] , as_index=False).agg({"Indice" : ["min","max"]})
Autor_range.columns = Autor_range.columns.droplevel(1)
Autor_range.columns = ["Author","MinIndex","MaxIndex"]

In [489]:
Autor_Unique = Autor.loc[: , ["Author"]].drop_duplicates()
dfFinal = Autor_Unique.merge(YearTotal.loc[: , ["Indice"]], how="cross")


In [490]:
dfFinal = dfFinal.merge(Autor_range, on = ["Author"], how="inner")


In [491]:
dfFinal = dfFinal.loc[ (dfFinal["Indice"] >= dfFinal["MinIndex"]) & (dfFinal["Indice"] <= dfFinal["MaxIndex"]) , ["Author","Indice"] ]


In [492]:
dfFinal = dfFinal.merge(YearTotal, on = ["Indice"] , how = "inner").loc[: , ["Author","Year"]]

In [493]:
dfFinal = dfFinal.merge(Autor, on=["Author","Year"] , how= "left").sort_values(["Author","Year"],ascending = True)


In [494]:
dfFinal = dfFinal.loc[: ,["Author","Year","Name"]].fillna(0)

In [495]:
dfFinal.columns = ['Author', 'Year', 'Conteo']
dfFinal['Conteo'] = dfFinal['Conteo'].astype('int')

In [496]:
dfFinal

,Author,Year,Conteo
0,Abraham Verghese,2010,1
50,Abraham Verghese,2011,1
439,Adam Gasiewski,2017,1
73,Adam Mansbach,2011,1
676,Adam Silvera,2021,1
...,...,...,...
141,Zhi Gang Sha,2009,1
49,Zhi Gang Sha,2010,0
102,Zhi Gang Sha,2011,0
176,Zhi Gang Sha,2012,0


## Probando datos

In [497]:
amazon.loc[amazon["Author"]=="Zhi Gang Sha", :]

,Name,Author,User Rating,Reviews,Price,Price_r,Year,Genre
96,Divine Soul Mind Body Healing and Transmission...,Zhi Gang Sha,4.6,37,$6.00,6,2009,Non Fiction
361,Soul Healing Miracles: Ancient and New Sacred ...,Zhi Gang Sha,4.6,220,$17.00,17,2013,Non Fiction


In [498]:
##Dataset final
dfFinal.loc[dfFinal["Author"]=="Zhi Gang Sha"]

,Author,Year,Conteo
141,Zhi Gang Sha,2009,1
49,Zhi Gang Sha,2010,0
102,Zhi Gang Sha,2011,0
176,Zhi Gang Sha,2012,0
227,Zhi Gang Sha,2013,1


In [499]:
dfFinal.loc[dfFinal["Author"]=="Stephenie Meyer"]

,Author,Year,Conteo
116,Stephenie Meyer,2009,6
37,Stephenie Meyer,2010,1
69,Stephenie Meyer,2011,0
154,Stephenie Meyer,2012,0
204,Stephenie Meyer,2013,0
260,Stephenie Meyer,2014,0
317,Stephenie Meyer,2015,0
375,Stephenie Meyer,2016,0
434,Stephenie Meyer,2017,0
494,Stephenie Meyer,2018,0


In [500]:
dfFinal.loc[dfFinal["Author"] == "Kathryn Stockett"]

,Author,Year,Conteo
112,Kathryn Stockett,2009,1
24,Kathryn Stockett,2010,1
63,Kathryn Stockett,2011,2


Que Autores han trabado en un año en mas de un libro

Ejemplo de salida esperada:

In [501]:
pd.DataFrame({
    
    "Autor" : ['Uayeb','Imix']
    , "Year" : [2010, 2010]
    , "Conteo de Titulos" : [2 , 3]
    
})

,Autor,Year,Conteo de Titulos
0,Uayeb,2010,2
1,Imix,2010,3


In [502]:
Autor_2 = dfFinal.copy()
Autor_2 = Autor_2.loc[Autor_2["Conteo"]>1, :].sort_values(["Year"], ascending = True)

In [503]:
Autor_2.sort_values(["Year"], ascending = True)

,Author,Year,Conteo
113,Malcolm Gladwell,2009,2
104,Charlaine Harris,2009,2
117,Stieg Larsson,2009,2
116,Stephenie Meyer,2009,6
126,David Zinczenko,2009,2
110,Jeff Kinney,2009,2
108,Glenn Beck,2009,2
39,Suzanne Collins,2010,3
35,Rick Riordan,2010,4
38,Stieg Larsson,2010,3


In [504]:
Autor_2.loc[Autor_2["Conteo"]<=1, :]

,Author,Year,Conteo


Que genero por año ha obtenido el mejor rating en promedio. (solo deberia de obtener un registro por año)

- ejemplo de la salida esperada:

In [505]:
pd.DataFrame({
    "Year" : [2009,2010]
    , "Genre" : [ "Fiction" , "Fiction" ]
    , "Mean User Rating": [ 4.59 , 4.61 ]
})

,Year,Genre,Mean User Rating
0,2009,Fiction,4.59
1,2010,Fiction,4.61


In [506]:
pd_genero = amazon.loc[: ,["Year","Genre","User Rating"]]
pd_genero = pd_genero.groupby(["Year","Genre"], as_index = False).agg({"User Rating": ["mean"]})
pd_genero.columns = pd_genero.columns.droplevel(1)

In [507]:
pd_genero["Count"]= pd_genero.sort_values(["Year","User Rating"] , ascending = False).groupby(["Year"]).cumcount()

In [508]:
pd_genero = pd_genero.loc[pd_genero["Count"]==0, ["Year","Genre","User Rating"]].sort_values(["Year"], ascending = True)

In [509]:
pd_genero

,Year,Genre,User Rating
0,2009,Fiction,4.591667
2,2010,Fiction,4.615000
4,2011,Fiction,4.619048
7,2012,Non Fiction,4.558621
9,2013,Non Fiction,4.561538
10,2014,Fiction,4.631034
12,2015,Fiction,4.652941
14,2016,Fiction,4.715789
16,2017,Fiction,4.737500
18,2018,Fiction,4.738095


Cuales son los autores que por lo menos dos años consecutivos hayan tenido en promedio ratings que decrecen 

Ejemplo

- Si Uayeb caballero, en 2010 saco dos libros, y el rating de esos dos libros es de 3.1, ese mismo autor en 2011 en su unico libro publicado ese año su promedio fue de 2.9 y en 2012 sus tres libros publicados en promedio su rating es de 2.8. Podemos ver que son dos años que este usuario ha ido decreciendo 

No importa que haya publicado en años salteados, ejemplo, 2010 si, 2011 no, 2012 no, 2013 si, 2014 si. para este caso el analisis comienza en 2010, 2013 y 2014

Ejemplo de salida esperada:

In [24]:
pd.DataFrame({ "autores" : ['Uayeb','Imix'] })

,autores
0,Uayeb
1,Imix


In [510]:
df_dec = amazon.copy()

In [511]:
df_dec = df_dec.loc[: , ["Name","Author","Year","User Rating"]].sort_values(["Author"], ascending = True)

In [512]:
df_dec = df_dec.groupby(["Author","Year"], as_index = False).agg({"User Rating" : ["mean"]})
df_dec.columns = df_dec.columns.droplevel(1)

In [513]:
df_dec["PrevYearRating"]=df_dec.groupby(["Author"], as_index = False)["User Rating"].shift(1)
df_dec["Prev2YearRating"]=df_dec.groupby(["Author"], as_index = False)["User Rating"].shift(2)
##df_dec["Minus"] = df_dec.apply(lambda r: r["User Rating"] - r["PrevYearRating"], axis = 1)
df_dec = df_dec.fillna(0)

In [514]:
df_dec["Flag"] = df_dec.apply(lambda r: 1 if (r["Prev2YearRating"] > r["PrevYearRating"] and r["PrevYearRating"] > r["User Rating"]) else 0, axis = 1)

In [515]:
df_dec = df_dec.loc[df_dec["Flag"]==1, ["Author"]]

In [516]:
df_dec

,Author
62,Bill O'Reilly
87,Charlaine Harris


Cuales son los libros que por año se encuentran arriba del percentil 70. (Sin importar genero)

Ejemplo de salida esperada

In [26]:
pd.DataFrame({
    "Year": [2010,2010,2011,2011]
    , "Titulos": ["Libro A","Libro B","Libro Z","Libro Y"]
    , "Autor": ["Autor 1","Autor 2","Autor 1","Autor 3"]
    , "Rating": [4.1, 4.0,4.2,4.3]
    , "Rating .70": [3.8,3.8,3.7,3.7]
})

,Year,Titulos,Autor,Rating,Rating .70
0,2010,Libro A,Autor 1,4.1,3.8
1,2010,Libro B,Autor 2,4.0,3.8
2,2011,Libro Z,Autor 1,4.2,3.7
3,2011,Libro Y,Autor 3,4.3,3.7


Este ejemplo es para que se haga una idea de la estructura de salida esperada, claramente por año dependiendo de las publicaciones pueden haber varios libros arriba del percentil 70


In [517]:
df_percentil = amazon.copy()

In [518]:
df_percentil = df_percentil.loc[: , ["Year","Name","Author","User Rating"]]

In [519]:
df_percentil.columns = ['Year', 'Name', 'Author', 'UserRating']

In [520]:
df_percentil2 = df_percentil.groupby(['Year'], as_index = False).UserRating.apply(lambda r: np.percentile(r, 70))

In [521]:
df_percentil2.columns = ['Year', 'Rating .70']

In [522]:
df_percentil = df_percentil.merge( df_percentil2 , on=["Year"] , how = "inner")

In [523]:
df_percentil = df_percentil.sort_values(["Year"] , ascending = True)

In [524]:
df_percentil = df_percentil.sort_values(["Year", "Name"], ascending = True)

In [525]:
df_percentil["Flag"] = df_percentil.apply(lambda r: 1 if r["UserRating"] > r["Rating .70"]  else 0, axis = 1)

In [526]:
df_percentil = df_percentil.loc[df_percentil["Flag"]==1,  ["Year","Name","Author","UserRating","Rating .70" ]]

In [527]:
df_percentil

,Year,Name,Author,UserRating,Rating .70
455,2009,Diary of a Wimpy Kid: The Last Straw (Book 3),Jeff Kinney,4.8,4.7
457,2009,"Dog Days (Diary of a Wimpy Kid, Book 4) (Volum...",Jeff Kinney,4.8,4.7
465,2009,Have a Little Faith: A True Story,Mitch Albom,4.8,4.7
467,2009,Liberty and Tyranny: A Conservative Manifesto,Mark R. Levin,4.8,4.7
468,2009,"Mastering the Art of French Cooking, Vol. 2",Julia Child,4.8,4.7
...,...,...,...,...,...
168,2021,I Love You to the Moon and Back,Amelia Hepworth,4.9,4.8
173,2021,"Oh, the Places You'll Go!",Dr. Seuss,4.9,4.8
187,2021,The Hill We Climb: An Inaugural Poem for the C...,Amanda Gorman,4.9,4.8
195,2021,The Very Hungry Caterpillar,Eric Carle,4.9,4.8
